# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import torch
import torch.utils.data
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.15.4
pandas version: 0.25.1


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [7]:
# should be the name of directory you created to save your features data
data_dir = 'data'

In [8]:
# take a look at some matsnu example domains
from dga import matsnu

for i in range(10):
    print(matsnu.generate_domain())

forcecrashhurryattemptconsider.com
clubhandcatchlimitedsmoke.com
afternoondoubtresolvecouple.com
organizationimaginephrase.com
homebearcheckservicerisk.com
eveningshipmatteroperate.com
gateexploreauthorbellperfect.com
bottlelookcommentshoulder.com
signatureplanbluewastewaste.com
extensioncrashfloorsense.com


In [11]:
# matsnu domains
matsnu_list = []

for i in range(20000):
    matsnu_list.append(matsnu.generate_domain())
    
matsnu_df = pd.DataFrame(matsnu_list, columns=['domain'])

print("Matsnu Shape:", matsnu_df.shape)

matsnu_df['label'] = 'dga'

matsnu_df.head()

Matsnu Shape: (20000, 1)


,domain,label
0,mediaauthortitletransition.com,dga
1,plantfixclosedcelebrateprompt.com,dga
2,accidentpeaklunchkidneck.com,dga
3,catchsmokescrewliftcontest.com,dga
4,examinationcrewassociate.com,dga


In [12]:
# alex top 1 million domains
alexa_df = pd.read_csv(data_dir + "/alexa_top_1m.csv", names=['domain'])

alexa_df['label'] = 'benign'

alexa_df.head()

,domain,label
1,google.com,benign
2,youtube.com,benign
3,facebook.com,benign
4,baidu.com,benign
5,wikipedia.org,benign


In [13]:
train_df = pd.concat([alexa_df.iloc[:20000], matsnu_df.iloc[:20000]], axis=0).reset_index(drop=True)

In [14]:
train_df.head()

,domain,label
0,google.com,benign
1,youtube.com,benign
2,facebook.com,benign
3,baidu.com,benign
4,wikipedia.org,benign


In [16]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
le = LabelEncoder()

In [17]:
# Apply le on categorical feature columns
train_df['label'] = le.fit_transform(train_df['label'])

train_df['label'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [19]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

# print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + string.digits + " .'"
n_letters = len(all_letters)

In [20]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('b'))

print(lineToTensor('blindpagegolucksinkorganize.com').size())

print(len('blindpagegolucksinkorganize.com'))

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([31, 1, 65])
31


In [22]:
import random

def randomTrainingExample():
    category = random.randint(0, 1)
    line = train_df[train_df['label'] == category].sample()['domain'].values[0]
    category_tensor = torch.tensor([category], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = 0 / line = mlit.go.jp
category = 0 / line = cineulagam.com
category = 1 / line = juiceassociatewarnchairisland.com
category = 1 / line = farmerweighwakereasondiet.com
category = 0 / line = beginnersbook.com
category = 1 / line = tillformrecommenddumpreward.com
category = 1 / line = videoemployoccasioncomplete.com
category = 0 / line = dubizzle.com
category = 1 / line = lockshelterfishscaledetailed.com
category = 1 / line = crackgrounddoctorreserve.com


In [21]:
# TO:DO Figure how this works in DLTK
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_model")
print(df.describe())
print(param)

## Stage 2 - create and initialize a model

In [ ]:
def init(df,param):
    
    X = df[param['feature_variables']]
    Y = df[param['target_variables']]
    
    input_size = int(X.shape[1])
    num_classes = len(np.unique(Y.to_numpy()))
    
    learning_rate = 0.001
    
    mapping = { key: value for value,key in enumerate(np.unique(Y.to_numpy().reshape(-1))) }
    
    # TODO: Change these
    print("FIT build RNN model with input shape " + str(X.shape))
    print("FIT build model with target classes " + str(num_classes))
    
    model = {
        "input_size": input_size,
        "num_classes": num_classes,
        "learning_rate": learning_rate,
        "mapping": mapping,
        "num_epochs": 10000,
        "batch_size": 100,
    }
    
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model['num_epochs'] = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model['batch_size'] = int(param['options']['params']['batch_size'])

    # RNN Layer
    model['rnn'] = nn.RNN(input_features, hidden_dim, n_layers, dropout=0.3)

    # Fully connected layer
    model['softmax'] = nn.LogSoftmax(dim=1)
    
    # Define loss and optimizer
    model['criterion'] = torch.nn.NLLLoss()
    
    model['optimizer'] = torch.optim.SGD(model['model'].parameters(), lr=learning_rate)
    
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model, df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    y_hat = np.ones(df.shape[0]) * model['hyperparameter']
    result = pd.DataFrame(y_hat, columns=['meaning_of_life'])
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code